# Exercise 02 : Join
## Required data

In [1]:
%ls ../data/checking-logs.sqlite

../data/checking-logs.sqlite


## Imports

In [2]:
import pandas as pd
import sqlite3

## Connect to DB

In [3]:
db_connection = sqlite3.connect('../data/checking-logs.sqlite')

## Create a new table `datamart` in the database by joining the tables `pageviews` and `checker` 
* the table should have the following columns: `uid, labname, first_commit_ts, first_view_ts`
* `first_commit_ts` is just a new name of the column timestamp from the checker table, it shows the first commit from a particular lab and from a particular user
* `first_view_ts` is the first visit of a user to the table `pageviews`, timestamp when a user visited the newsfeed
* `status = ’ready’` should still be a filter 
* `numTrials = 1` should still be a filter 
* `labnames` should still be from the list: `’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’`
* the table should contain only the users (`uid`s with `user_*`) and not the admins 
* `first_commit_ts` and `first_view_ts` should be parsed as `datetime64[ns]`

In [4]:
sql_query="""
CREATE TABLE datamart AS
    SELECT 
        checker_filtered.uid,
        checker_filtered.labname,
        checker_filtered.first_commit_ts,
        pageviews_filtered.first_view_ts
    FROM
    (
        SELECT 
            uid, labname, timestamp AS first_commit_ts
        FROM 
            checker
        WHERE
            uid LIKE 'user_%'
            AND status = 'ready'
            AND numTrials = 1
            AND labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    ) checker_filtered
    LEFT OUTER JOIN 
    (
        SELECT 
            uid, MIN(datetime) as first_view_ts
        FROM 
            pageviews
        GROUP BY
            uid
    ) pageviews_filtered
    ON
        checker_filtered.uid = pageviews_filtered.uid
"""

cur = db_connection.cursor()
cur.execute('DROP TABLE IF EXISTS datamart;')
cur.execute(sql_query)

In [5]:
datamart = pd.read_sql(sql='SELECT * FROM datamart;', con=db_connection,
                       parse_dates=['first_commit_ts', 'first_view_ts']
                      )
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


In [6]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


## Dataframe `test` with the users that have the values in `first_view_ts`

In [7]:
test = datamart[datamart['first_view_ts'].notnull()]
test

,uid,labname,first_commit_ts,first_view_ts
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


## Dataframe `control` with the users that have missing values in `first_view_ts`

In [9]:
control = datamart[datamart['first_view_ts'].isnull()]
control

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
5,user_2,laba04,2020-04-18 13:42:35.482008,NaT
6,user_2,laba04s,2020-04-18 13:51:22.291271,NaT
...,...,...,...,...
126,user_2,laba06s,2020-05-19 14:45:03.908268,NaT
132,user_6,laba06s,2020-05-20 14:50:07.609937,NaT
134,user_7,laba06s,2020-05-20 23:05:37.742597,NaT
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT


## Replace the missing values in the `control` with the average `first_view_ts` of the `test` users

In [10]:
control = control.fillna(test['first_view_ts'].mean())
control

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783552
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783552
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783552
5,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783552
6,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783552
...,...,...,...,...
126,user_2,laba06s,2020-05-19 14:45:03.908268,2020-04-27 00:40:05.761783552
132,user_6,laba06s,2020-05-20 14:50:07.609937,2020-04-27 00:40:05.761783552
134,user_7,laba06s,2020-05-20 23:05:37.742597,2020-04-27 00:40:05.761783552
135,user_23,laba06,2020-05-21 08:34:10.517205,2020-04-27 00:40:05.761783552


In [11]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


## Save both tables into the database

In [12]:
test.to_sql('test', db_connection, if_exists='replace')
control.to_sql('control', db_connection, if_exists='replace')

81

In [13]:
pd.read_sql('SELECT * FROM test;', db_connection)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [14]:
pd.read_sql('SELECT * FROM control;', db_connection)

,index,uid,labname,first_commit_ts,first_view_ts
0,0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783
1,1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783
2,2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783
3,5,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783
4,6,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783
...,...,...,...,...,...
76,126,user_2,laba06s,2020-05-19 14:45:03.908268,2020-04-27 00:40:05.761783
77,132,user_6,laba06s,2020-05-20 14:50:07.609937,2020-04-27 00:40:05.761783
78,134,user_7,laba06s,2020-05-20 23:05:37.742597,2020-04-27 00:40:05.761783
79,135,user_23,laba06,2020-05-21 08:34:10.517205,2020-04-27 00:40:05.761783


## Close the connection

In [15]:
db_connection.close()